<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4df97c85ef58ece90c6154ef8581d8baa5bbc7b328611630d6430cb228460070
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-31 09:29:33
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 32.54 K (0.22%)
Current PnL: -18.18 L (-11.92%)
CY Booked + Current PnL: -6.05 L (-3.97%)
-------------------
Total profit:  2.94 L
Total loss:  -21.12 L
-------------------
Total Booked + Current PnL: 20.93 L (16.66%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.26 L (58.39%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.91,70.0,X-LC,4.95,233470.0,23104.0,5673.0,0.54,10.98,2.43,13.68,37.0,4.07,1.62,29.32,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.50,89656.0,-11121.0,11216.0,0.55,-11.04,12.51,0.09,245.0,-0.99,0.62,17.79,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-16.77,71.0,H-MC,17.61,215074.0,35266.0,14087.0,0.72,19.61,6.55,27.45,88.0,2.50,1.49,60.39,XR,NTT,BANKS
42,ITC,452.0,-36.23,64.0,X-LC,1.22,205747.0,5609.0,15287.0,0.48,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-10.05,40.0,X-MC,2.32,88950.0,-1728.0,16634.0,0.17,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.0,-2.53,78.0,X-LC,4.50,255661.0,21517.0,27100.0,6.58,9.19,10.60,20.77,86.0,0.79,1.77,16.74,X40N,NTT,BREWERIES
56,RAJOOENG,143.1,-39.52,54.0,H-SC,16.72,91570.0,-10930.0,51526.0,3.07,-10.66,56.27,39.61,114.0,-0.21,0.63,11.83,AR,ATH,MISC
70,STARHEALTH,761.0,21.02,61.0,H-SC,13.48,269443.0,-11680.0,144529.0,2.96,-4.15,53.64,47.26,144.0,-0.08,1.86,44.36,XY24,NTT,INSURANCE
80,UNIONBANK,163.0,-8.53,61.0,M-LC,9.86,166593.0,25753.0,18742.0,2.95,18.29,11.25,31.59,66.0,1.37,1.15,49.62,XY24,NTT,BANKS
63,SAMMAANCAP,326.0,-198.86,76.0,M-SC,30.27,169983.0,20763.0,123425.0,2.44,13.91,72.61,96.62,208.0,0.17,1.18,83.96,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,400.00,-13.31,44.0,H-SC,3.50,217616.0,-60943.0,318394.0,-4.80,-21.88,146.31,92.42,151.0,-0.19,1.51,25.76,XY24,NTT,BANKS
84,VBL,671.64,-15.37,57.0,X-LC,7.94,304677.0,-11165.0,123821.0,-1.59,-3.53,40.64,35.67,5.0,-0.09,2.11,9.83,X40N,ATH,FMCG
22,DABUR,735.00,-11.40,42.0,X-MC,3.44,239663.0,-8405.0,116812.0,-1.48,-3.39,48.74,43.70,102.0,-0.07,1.66,10.57,XY24,BTT,FMCG
27,HAPPSTMNDS,1488.71,-23.29,47.0,H-SC,16.83,82728.0,-45437.0,155462.0,-0.82,-35.45,187.92,85.85,132.0,-0.29,0.57,5.61,AR,ATH,IT
48,LAOPALA,464.00,98.39,41.0,H-SC,3.29,90893.0,-40212.0,91457.0,-0.79,-30.67,100.62,39.09,142.0,-0.44,0.63,17.06,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.0,-14.25,46.0,M-LC,5.28,150505.0,-440.0,109899.0,0.34,-0.29,73.02,72.51,53.0,-0.0,1.04,5.71,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.50,89656.0,-11121.0,11216.0,0.55,-11.04,12.51,0.09,245.0,-0.99,0.62,17.79,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,53.0,H-SC,2.87,226080.0,-43587.0,79919.0,0.28,-16.16,35.35,13.47,138.0,-0.55,1.56,14.96,XY24,NTT,PAINTS
18,CERA,9475.0,-21.96,55.0,H-SC,2.30,142462.0,-33441.0,75462.0,0.17,-19.01,52.97,23.89,149.0,-0.44,0.99,23.03,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,6.12,105165.0,-26670.0,68894.0,-0.08,-20.23,65.51,32.03,98.0,-0.39,0.73,17.79,OX40N,NTT,IT
68,SIS,528.0,2020.73,43.0,H-SC,3.27,85578.0,-25454.0,49062.0,-0.42,-22.92,57.33,21.26,156.0,-0.52,0.59,15.62,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,48.36,57.0,M-MC,10.28,234381.0,9419.0,158231.0,-0.69,4.19,67.51,74.53,192.0,0.06,1.62,37.79,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,57.0,M-LC,5.17,209055.0,5885.0,36250.0,0.48,2.90,17.34,20.74,55.0,0.16,1.45,9.96,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.88,49.0,M-LC,11.29,208365.0,3289.0,109287.0,0.29,1.60,52.45,54.89,52.0,0.03,1.44,26.09,X5K,ATH,METALS
47,KPIGREEN,731.05,12.48,72.0,H-SC,5.25,133837.0,8540.0,50390.0,-0.62,6.82,37.65,47.03,141.0,0.17,0.93,67.58,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,61.0,M-LC,9.86,166593.0,25753.0,18742.0,2.95,18.29,11.25,31.59,66.0,1.37,1.15,49.62,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-16.77,71.0,H-MC,17.61,215074.0,35266.0,14087.0,0.72,19.61,6.55,27.45,88.0,2.50,1.49,60.39,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-36.62,68.0,M-SC,4.08,131505.0,16707.0,77338.0,0.60,14.55,58.81,81.92,223.0,0.22,0.91,55.68,XR,NTT,DURABLES
47,KPIGREEN,731.05,12.48,72.0,H-SC,5.25,133837.0,8540.0,50390.0,-0.62,6.82,37.65,47.03,141.0,0.17,0.93,67.58,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130433.0,7097.0,124524.0,-0.35,5.75,95.47,106.72,119.0,0.06,0.90,30.48,AR,ATH,MISC
32,HINDZINC,730.22,27.88,49.0,M-LC,11.29,208365.0,3289.0,109287.0,0.29,1.60,52.45,54.89,52.0,0.03,1.44,26.09,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,20.0,X-LC,22.16,149719.0,-126804.0,236870.0,0.27,-45.86,158.21,39.81,54.0,-0.54,1.04,5.53,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.44,29.0,X-SC,5.34,89100.0,-39570.0,84868.0,0.14,-30.75,95.25,35.20,219.0,-0.47,0.62,2.27,X40,NTT,FOOTWEAR
24,DMART,5391.45,-18.21,33.0,X-LC,9.35,100090.0,-2496.0,29306.0,0.22,-2.43,29.28,26.13,38.0,-0.09,0.69,22.52,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.89,36.0,X-MC,5.12,198518.0,-1829.0,32339.0,-0.44,-0.91,16.29,15.23,178.0,-0.06,1.37,31.04,X40N,NTT,REALTY
31,HINDUNILVR,2922.00,-11.98,38.0,X-LC,7.41,247150.0,-9913.0,45055.0,0.08,-3.86,18.23,13.67,24.0,-0.22,1.71,15.77,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-36.23,64.0,X-LC,1.22,205747.0,5609.0,15287.0,0.48,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
67,SIEMENS,4671.50,-2.61,48.0,H-LC,1.23,156570.0,-29525.0,77001.0,0.15,-15.87,49.18,25.51,15.0,-0.38,1.08,15.38,AR,ATH,ELECTRICAL
53,PGHH,17907.65,-30.41,39.0,X-MC,2.01,201300.0,480.0,67315.0,0.05,0.24,33.44,33.76,80.0,0.01,1.39,5.45,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.19,51.0,X-MC,2.86,316281.0,-11710.0,118257.0,-0.01,-3.57,37.39,32.48,92.0,-0.10,2.19,7.45,X40,ATH,ELECTRICAL
81,UNITDSPR,1644.00,-2.53,78.0,X-LC,4.50,255661.0,21517.0,27100.0,6.58,9.19,10.60,20.77,86.0,0.79,1.77,16.74,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.44,29.0,X-SC,5.34,89100.0,-39570.0,84868.0,0.14,-30.75,95.25,35.20,219.0,-0.47,0.62,2.27,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.42,41.0,X-MC,6.75,81960.0,-1550.0,21580.0,0.09,-1.86,26.33,23.98,82.0,-0.07,0.57,3.86,X40,ATH,APPARELS
54,QUESS,986.00,-46.01,47.0,X-SC,37.78,53480.0,-11526.0,162456.0,-0.75,-17.73,303.77,232.18,198.0,-0.07,0.37,4.31,XY24,NTT,MISC
21,COLPAL,3767.14,-3.66,48.0,X-MC,7.49,224830.0,-38535.0,151895.0,-0.31,-14.63,67.56,43.04,84.0,-0.25,1.56,4.38,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-14.06,49.0,X-MC,5.92,181302.0,-19204.0,116360.0,-0.15,-9.58,64.18,48.45,90.0,-0.17,1.25,4.55,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.01,47.0,X-SC,37.78,53480.0,-11526.0,162456.0,-0.75,-17.73,303.77,232.18,198.0,-0.07,0.37,4.31,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.42,41.0,X-MC,6.75,81960.0,-1550.0,21580.0,0.09,-1.86,26.33,23.98,82.0,-0.07,0.57,3.86,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.39,46.0,X-SC,4.65,82682.0,-62478.0,138401.0,-0.33,-43.04,167.39,52.31,136.0,-0.45,0.57,10.67,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,40.0,X-MC,2.32,88950.0,-1728.0,16634.0,0.17,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.44,29.0,X-SC,5.34,89100.0,-39570.0,84868.0,0.14,-30.75,95.25,35.20,219.0,-0.47,0.62,2.27,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,10.57,288016.0,-57940.0,124653.0,0.95,-16.75,43.28,19.28,1.0,-0.46,1.99,6.48,X40,ATH,IT
40,INFY,2275.00,-18.16,56.0,X-LC,5.49,318627.0,5739.0,165941.0,0.14,1.83,52.08,54.87,3.0,0.03,2.20,10.27,X40,BTT,IT
42,ITC,452.00,-36.23,64.0,X-LC,1.22,205747.0,5609.0,15287.0,0.48,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
84,VBL,671.64,-15.37,57.0,X-LC,7.94,304677.0,-11165.0,123821.0,-1.59,-3.53,40.64,35.67,5.0,-0.09,2.11,9.83,X40N,ATH,FMCG
1,ABB,7934.00,-39.38,56.0,H-LC,5.44,253488.0,-8131.0,127352.0,0.04,-3.11,50.24,45.57,7.0,-0.06,1.75,7.86,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7022.22,58.0,L-SC,8.27,78173.0,-15637.0,92252.0,0.46,-16.67,118.01,81.67,269.0,-0.17,0.54,51.39,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.33,81216.0,-32333.0,72388.0,0.45,-28.47,89.13,35.27,268.0,-0.45,0.56,98.78,XR,NTT,HOTELS
79,TTKPRESTIG,770.00,108.11,60.0,M-SC,1.50,89656.0,-11121.0,11216.0,0.55,-11.04,12.51,0.09,245.0,-0.99,0.62,17.79,OX40N,NTT,DURABLES
50,MASFIN,398.61,-18.70,53.0,H-SC,8.76,92475.0,-5505.0,27104.0,-0.05,-5.62,29.31,22.05,152.0,-0.20,0.64,34.42,XR,ATH,FINANCE
37,INDIAMART,4810.62,-53.33,55.0,H-SC,6.52,130433.0,7097.0,124524.0,-0.35,5.75,95.47,106.72,119.0,0.06,0.90,30.48,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,77.41,75.0,H-SC,2.18,153600.0,-29175.0,140237.0,-0.61,-15.96,91.30,60.77,125.0,-0.21,1.06,37.57,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,12.48,72.0,H-SC,5.25,133837.0,8540.0,50390.0,-0.62,6.82,37.65,47.03,141.0,0.17,0.93,67.58,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-36.62,68.0,M-SC,4.08,131505.0,16707.0,77338.0,0.60,14.55,58.81,81.92,223.0,0.22,0.91,55.68,XR,NTT,DURABLES
69,SONACOMS,806.63,-31.67,72.0,M-SC,7.49,85094.0,-16065.0,56068.0,0.68,-15.88,65.89,39.54,202.0,-0.29,0.59,19.99,AR,ATH,AUTO
63,SAMMAANCAP,326.00,-198.86,76.0,M-SC,30.27,169983.0,20763.0,123425.0,2.44,13.91,72.61,96.62,208.0,0.17,1.18,83.96,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.66
1,25,43.98
2,50,74.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.11
LC    32.38
MC    24.49
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.95
X40      16.20
XY25     12.06
XR       11.46
X40N     10.71
AR        7.83
OX40N     6.56
X200      1.78
X5K       1.44
SR        1.06
MH        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.44
X-LC    21.69
X-MC    18.68
M-SC    12.72
M-LC     5.08
H-LC     4.61
X-SC     4.51
H-MC     3.87
M-MC     1.62
L-SC     1.44
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.43,-3.33,37.05
IT,12.47,-16.50,78.98
FINANCE,12.15,-7.64,56.43
MISC,6.83,-16.15,80.50
ELECTRICAL,5.79,-7.85,46.09
BANKS,5.58,-6.22,61.15
PAINTS,5.49,-16.27,33.60
INSURANCE,3.83,0.21,36.99
AUTO,2.77,-43.79,103.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243206.0,22
XR,1228653.0,14
AR,1155147.0,9
X40,815173.0,12
X40N,609316.0,9
OX40N,538440.0,9
XY25,442174.0,8
SR,267616.0,2
X5K,109287.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3298818.0,24
M-SC,1206013.0,15
X-MC,1177777.0,14
X-LC,1003223.0,13
X-SC,527714.0,6
M-LC,274178.0,4
H-LC,270469.0,3
H-MC,258835.0,3
L-SC,253448.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1153091.0      6
           AR         828758.0      5
           XR         758504.0      7
M-SC       XY24       695686.0      6
X-MC       XY24       576095.0      4
X-LC       X40        446040.0      5
           XY24       303051.0      2
X-MC       X40        284265.0      6
X-SC       X40N       280390.0      4
H-SC       SR         267616.0      2
           OX40N      240459.0      3
H-LC       AR         204353.0      2
X-LC       X40N       180227.0      3
H-MC       XY24       175854.0      1
X-MC       XY25       168718.0      2
L-SC       XR         164640.0      2
X-SC       XY24       162456.0      1
M-MC       XY24       158231.0      1
X-MC       X40N       148699.0      2
M-SC       OX40N      140279.0      4
           XR         124587.0      2
           XY25       123425.0      1
           AR         122036.0      2
M-LC       XR         109899.0      1
           X5K        109287.0      1
L-SC       OX40N       88808.0      1
X-SC       X40         84868.0      1
X-LC       XY25        73905.0      3
H-MC       OX40N       68894.0      1
H-LC       X200        66116.0      1
L-MC       XR          56936.0      1
H-SC       MH          50390.0      1
L-LC       XY25        39876.0      1
M-LC       XY25        36250.0      1
           XY24        18742.0      1
H-MC       XR          14087.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
